<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2)-Data-cleaning-and-preprocessing" data-toc-modified-id="2)-Data-cleaning-and-preprocessing-1">2) Data cleaning and preprocessing</a></span><ul class="toc-item"><li><span><a href="#Improve-readability" data-toc-modified-id="Improve-readability-1.1">Improve readability</a></span></li><li><span><a href="#Extract-useful-information-from-the-time-stamp" data-toc-modified-id="Extract-useful-information-from-the-time-stamp-1.2">Extract useful information from the time stamp</a></span></li><li><span><a href="#Check-the-data-types-and-classes" data-toc-modified-id="Check-the-data-types-and-classes-1.3">Check the data types and classes</a></span></li><li><span><a href="#Filling-in-missing-values" data-toc-modified-id="Filling-in-missing-values-1.4">Filling in missing values</a></span></li><li><span><a href="#Aggregate-categories-with-few-occurrences" data-toc-modified-id="Aggregate-categories-with-few-occurrences-1.5">Aggregate categories with few occurrences</a></span></li></ul></li></ul></div>

## 2) Data cleaning and preprocessing

In [19]:
import pandas as pd

In [20]:
pirate_data = pd.read_csv(f"data/pirate_data.csv", index_col = 0)
pirate_data.head()

,DATETIME (LOCAL),DATETIME (UTC),DATE (LT),DATE (UTC),REGION,COUNTRY,INCIDENT TYPE,VESSEL TYPE,MAERSK?,VESSEL ACTIVITY LOCATION,LAT,LONG,TIMEZONE,TIME OF DAY,ATTACKS,ATTACK SUCCESS
1,2016-01-30 23:34:00,2016-01-30 22:34:00,2016-01-30,2016-01-30,WEST AFRICA,NIGERIA,"FAILED ATTACK, HIJACKING",CONTAINER SHIP,0,OFF SHORE,3.000000,6.250000,Africa/Lagos,EVENING,1,0
2,2016-01-29 23:40:00,2016-01-29 22:40:00,2016-01-29,2016-01-29,WEST AFRICA,NIGERIA,KIDNAP/RANSOM AT SEA,PRODUCT TANKER,0,OFF SHORE,2.500000,5.750000,Africa/Lagos,EVENING,1,1
3,2016-01-28 03:00:00,2016-01-28 02:00:00,2016-01-28,2016-01-28,SOUTH ASIA,INDIA,"FAILED ATTACK, ROBBERY",PRODUCT TANKER,0,ANCHORAGE,22.816667,70.116667,Asia/Kolkata,NIGHT,1,0
4,NaN,NaN,2016-01-27,2016-01-27,HORN OF AFRICA/ GULF OF ADEN,SOMALIA,SUSPICIOUS ACTIVITY AT SEA,NaN,0,OFF SHORE,13.998747,54.112792,Asia/Riyadh,UNKNOWN,1,0
5,2016-01-25 23:55:00,2016-01-25 22:55:00,2016-01-25,2016-01-25,SOUTH AMERICA,VENEZUELA,THEFT AT ANCHORAGE,GENERAL CARGO VESSEL,0,ANCHORAGE,10.267500,-63.432500,America/Caracas,EVENING,1,1


### Improve readability

In [21]:
# Turn all caps into title or capitalized spelling
pirate_data["REGION"] = pirate_data["REGION"].str.title()
pirate_data["COUNTRY"] = pirate_data["COUNTRY"].str.title()
pirate_data["INCIDENT TYPE"] = pirate_data["INCIDENT TYPE"].str.capitalize()
pirate_data["VESSEL TYPE"] = pirate_data["VESSEL TYPE"].str.capitalize()
pirate_data["VESSEL ACTIVITY LOCATION"] = pirate_data["VESSEL ACTIVITY LOCATION"].str.capitalize()
pirate_data["TIME OF DAY"] = pirate_data["TIME OF DAY"].str.title()

# Sort by date
pirate_data = pirate_data.sort_values(["DATE (LT)"])
print(f"The data ranges from {pirate_data['DATE (LT)'].min()} to {pirate_data['DATE (LT)'].max()}.")

The data ranges from 2015-01-04 to 2016-06-29.


### Extract useful information from the time stamp

In [22]:
# We need only one type of date; remove three out of four.
pirate_data = pirate_data.drop(columns = ["DATETIME (LOCAL)","DATETIME (UTC)", "DATE (UTC)"])
pirate_data['DATE (LT)'] = pd.to_datetime(pirate_data['DATE (LT)'])

# Extract day, month etc. from timestamp
pirate_data['YEAR'] = pirate_data['DATE (LT)'].dt.year
pirate_data['MONTH'] = pirate_data['DATE (LT)'].dt.month
pirate_data['WEEK'] = pirate_data['DATE (LT)'].dt.isocalendar().week
pirate_data['DAY'] = pirate_data['DATE (LT)'].dt.day
pirate_data['WEEKDAY'] = pirate_data['DATE (LT)'].dt.dayofweek
pirate_data['YEAR_MONTH'] = pirate_data['DATE (LT)'].dt.strftime('%Y-%m')

In [23]:
pirate_data.head()

,DATE (LT),REGION,COUNTRY,INCIDENT TYPE,VESSEL TYPE,MAERSK?,VESSEL ACTIVITY LOCATION,LAT,LONG,TIMEZONE,TIME OF DAY,ATTACKS,ATTACK SUCCESS,YEAR,MONTH,WEEK,DAY,WEEKDAY,YEAR_MONTH
703,2015-01-04,Horn Of Africa/ Gulf Of Aden,Djibouti,Suspicious activity at sea,Merchant vessel,0,Off shore,12.553333,43.426667,Africa/Nairobi,Unknown,1,0,2015,1,1,4,6,2015-01
702,2015-01-04,South East Asia,Singapore,Theft at sea,Tug,0,Off shore,1.324500,104.541500,Asia/Kuala_Lumpur,Morning,1,1,2015,1,1,4,6,2015-01
701,2015-01-04,North Africa,Libya,Other maritime risk,Crude oil tanker,0,Anchorage,32.780116,22.663078,Africa/Tripoli,Afternoon,1,1,2015,1,1,4,6,2015-01
700,2015-01-06,Horn Of Africa/ Gulf Of Aden,Djibouti,Suspicious activity at sea,Merchant vessel,0,Off shore,13.253333,42.956667,Asia/Riyadh,Afternoon,1,0,2015,1,2,6,1,2015-01
699,2015-01-07,West Africa,Nigeria,Kidnap/ransom at anchorage,NaN,0,Anchorage,4.638297,6.812897,Africa/Lagos,Unknown,1,1,2015,1,2,7,2,2015-01


### Check the data types and classes

In [24]:
pirate_data.dtypes

DATE (LT)                   datetime64[ns]
REGION                              object
COUNTRY                             object
INCIDENT TYPE                       object
VESSEL TYPE                         object
MAERSK?                              int64
VESSEL ACTIVITY LOCATION            object
LAT                                float64
LONG                               float64
TIMEZONE                            object
TIME OF DAY                         object
ATTACKS                              int64
ATTACK SUCCESS                       int64
YEAR                                 int64
MONTH                                int64
WEEK                                UInt32
DAY                                  int64
WEEKDAY                              int64
YEAR_MONTH                          object
dtype: object

In [25]:
pirate_data['ATTACK SUCCESS'].value_counts()
# The data set is not well balanced.

1    500
0    302
Name: ATTACK SUCCESS, dtype: int64

### Filling in missing values

In [26]:
pirate_data.isna().sum()
# There is some missing data.

DATE (LT)                     0
REGION                        0
COUNTRY                       0
INCIDENT TYPE                 0
VESSEL TYPE                 164
MAERSK?                       0
VESSEL ACTIVITY LOCATION    126
LAT                           0
LONG                          0
TIMEZONE                      0
TIME OF DAY                   0
ATTACKS                       0
ATTACK SUCCESS                0
YEAR                          0
MONTH                         0
WEEK                          0
DAY                           0
WEEKDAY                       0
YEAR_MONTH                    0
dtype: int64

In [27]:
pirate_data['VESSEL TYPE'] = pirate_data['VESSEL TYPE'].fillna("Unknown")

In [28]:
pirate_data['VESSEL ACTIVITY LOCATION'] = pirate_data['VESSEL ACTIVITY LOCATION'].fillna("Unknown")

### Aggregate categories with few occurrences

In [29]:
pirate_data['VESSEL ACTIVITY LOCATION'].value_counts()
# This is fine.

Off shore    421
Anchorage    198
Unknown      126
Berth         57
Name: VESSEL ACTIVITY LOCATION, dtype: int64

In [30]:
# Reduce the number of vessel types
pirate_data["VESSEL TYPE"] = pirate_data["VESSEL TYPE"].replace("Unknown", "Unknown vessel type")
pirate_data["VESSEL TYPE"] = pirate_data["VESSEL TYPE"].replace(["Fishing trawler", "Trawler", "Fishing dhow"], 
                                                                "Fishing vessel") 
pirate_data["VESSEL TYPE"] = pirate_data["VESSEL TYPE"].replace(["Lng tanker", "Lng carrier", "Lpg tanker"], 
                                                                "Gas tanker")
pirate_data["VESSEL TYPE"] = pirate_data["VESSEL TYPE"].replace([
    'Vehicle carrier', 'Yacht', 'Other vessel type', 'Schooner', 'Small craft', 'Skipper', 'Reefer', 'Barge',
    'Floating storage offloading vessel', 'Wood chips carrier', 'Dredger', 'Crew vessel', 'Drillship', 
    'Heavy lift vessel', 'Cable layer', '', 'Motor vessel', 'Ore carrier', 'Pipe-layer', 'Oil rig', 
    'Research vessel', 'Security vessel', 'Ro-ro vessel', 'Surveillance vessel', 'Local craft', 
    'Bunker barge', 'Naval vessel'], "Other")


pirate_data['VESSEL TYPE'].value_counts()

Unknown vessel type     174
Bulk carrier            134
Product tanker          107
Container ship           60
Other                    49
Fishing vessel           46
Tug                      34
Tanker                   34
Merchant vessel          30
Crude oil tanker         28
General cargo vessel     28
Supply vessel            25
Passenger vessel         25
Chemical tanker          15
Gas tanker               13
Name: VESSEL TYPE, dtype: int64

In [31]:
# Clean up the regions
pirate_data["REGION"] = pirate_data["REGION"].replace(["North America", "South America", 
                                                         "Caribbean"], "Americas")
pirate_data["REGION"] = pirate_data["REGION"].replace(["Europe", "Middle East"], "Eastern Mediterranean")
pirate_data["REGION"] = pirate_data["REGION"].replace("Horn Of Africa/ Gulf Of Aden", "East Africa")
pirate_data["REGION"] = pirate_data["REGION"].replace("Pacific", "South East Asia")

pirate_data['REGION'].value_counts()
# One occurrence in India is still classified as 'East Africa'?!

South East Asia          316
West Africa              288
South Asia                67
East Africa               42
Americas                  31
Eastern Mediterranean     30
North Africa              18
North East Asia           10
Name: REGION, dtype: int64

In [32]:
pirate_data['COUNTRY'].unique()
# This feature is probably too granular for the model, but useful for visualization.

array(['Djibouti', 'Singapore', 'Libya', 'Nigeria', 'Indonesia',
       'Colombia', 'Iraq', 'Ghana', "Cote D'Ivoire", 'Liberia', 'Angola',
       'Ukraine', 'Oman', 'Vietnam', 'Malaysia', 'India', 'Benin', 'Togo',
       'Honduras', 'Brazil', 'Papua New Guinea', 'Sierra Leone', 'Guinea',
       'Guinea-Bissau', 'Cameroon', 'Senegal', 'Bangladesh',
       'Philippines', 'Somalia', 'Congo', 'Yemen', 'Thailand', 'Iran',
       'Trinidad And Tobago', 'Panama', 'Mozambique', 'Kenya', 'Egypt',
       'China', 'Venezuela', 'Bahrain', 'South Africa', 'South Korea',
       'Tanzania', 'Finland', 'Usa', 'Haiti', 'New Zealand', 'Turkey',
       'Australia', 'Peru', 'Guyana', 'Equatorial Guinea', 'Argentina',
       'Bahamas'], dtype=object)

In [33]:
pirate_data['TIMEZONE'].value_counts().head(15)
# Too many categories and we already have information on the region. We can drop this column.

Africa/Lagos         226
Asia/Jakarta         155
Asia/Dhaka            39
Asia/Kuala_Lumpur     39
Asia/Ho_Chi_Minh      35
Asia/Kolkata          29
Africa/Abidjan        27
Asia/Riyadh           24
Asia/Manila           23
Asia/Pontianak        21
Africa/Accra          20
Africa/Tripoli        19
Asia/Singapore        18
Africa/Nairobi        14
Asia/Makassar         12
Name: TIMEZONE, dtype: int64

In [34]:
pirate_data = pirate_data.drop(columns = "TIMEZONE")

In [35]:
# Remaining columns with data types:
pirate_data.dtypes

DATE (LT)                   datetime64[ns]
REGION                              object
COUNTRY                             object
INCIDENT TYPE                       object
VESSEL TYPE                         object
MAERSK?                              int64
VESSEL ACTIVITY LOCATION            object
LAT                                float64
LONG                               float64
TIME OF DAY                         object
ATTACKS                              int64
ATTACK SUCCESS                       int64
YEAR                                 int64
MONTH                                int64
WEEK                                UInt32
DAY                                  int64
WEEKDAY                              int64
YEAR_MONTH                          object
dtype: object

In [36]:
pirate_data.to_csv("data/pirate_data_cleaned.csv")